In [5]:
import os
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from src import image
from copy import deepcopy as copy
from LRFutils import color

First, we compute the shift of each images relatively to the first one. To do so, we take a reference star and get the approximative position. We define a sufficiently big box around it where the star should appear on all images. Finally, we run the algorithme that detect the source and we compare the coordinates to deduce the shift.

In [6]:
PARENT_PATH = "rsrc/8_Oct/" #@param {type:"string"}
X = 1083 #@param {type:"integer"}
Y = 660 #@param {type:"integer"}
box_size = 200 #@param {type:"integer"}
plot = False #@param {type:"boolean"}
print_results = True #@param {type:"boolean"}

positions = []
shifts = []
files = []

if print_results: print("Image shifts:")

nbfiles = 0
for file in os.listdir(PARENT_PATH):
    path = os.path.join(PARENT_PATH, file)
    if file.startswith("td10r"):
        nbfiles += 1
        files.append(file)

        positions.append(
            image.find_object(
                path,
                Xcent=X,
                Ycent=Y,
                box_size=box_size,
                plot=plot
            )
        )

        shifts.append(copy(positions[-1]))
        shifts[-1][0] -= positions[0][0]
        shifts[-1][1] -= positions[0][1]
        if print_results: print(f"    {file}: x={shifts[-1][0]:.3f}, y={shifts[-1][1]:.3f}")

positions = np.array(positions)
shifts = np.array(shifts)

Image shifts:
    td10r_n2_0096.fits: x=0.000, y=0.000
    td10r_n2_0098.fits: x=0.251, y=0.277
    td10r_n2_0100.fits: x=0.528, y=0.241
    td10r_n2_0102.fits: x=1.075, y=0.324
    td10r_n2_0104.fits: x=0.770, y=0.401
    td10r_n2_0106.fits: x=1.086, y=0.422
    td10r_n2_0108.fits: x=-70.855, y=-4.779
    td10r_n2_0110.fits: x=-70.739, y=-4.672
    td10r_n2_0112.fits: x=-70.540, y=-4.728
    td10r_n2_0114.fits: x=-70.542, y=-4.754
    td10r_n2_0116.fits: x=-75.443, y=36.101
    td10r_n2_0118.fits: x=-75.346, y=36.117
    td10r_n2_0123.fits: x=-30.407, y=-6.539
    td10r_n2_0125.fits: x=-29.995, y=-6.062
    td10r_n2_0127.fits: x=-29.862, y=-6.142
    td10r_n2_0129.fits: x=-29.666, y=-6.033
    td10r_n2_0131.fits: x=-28.322, y=-47.524
    td10r_n2_0133.fits: x=-27.555, y=-47.885
    td10r_n2_0135.fits: x=-61.413, y=-46.751
    td10r_n2_0137.fits: x=-61.186, y=-46.692
    td10r_n2_0144.fits: x=-46.295, y=17.827
    td10r_n2_0146.fits: x=-46.212, y=17.954
    td10r_n2_0148.fits: x=-46.09

Once we have theses shifts, we can compute the position of the asteroid on each image by deducing it from the position on the first image and the one on the last image. We remove the shift of the last image to have the position of the object if the telescope was perfectly aligned.

Then, we have an almost straight line and we can ddeduce the position of each image by looking at the long of the line.

In [7]:
first_X = 1150 #@param {type:"integer"}
first_Y = 1157 #@param {type:"integer"}  
last_X = 1172 #@param {type:"integer"}
last_Y = 1104 #@param {type:"integer"}

first_position = [1150,1157]
last_position = [1172-shifts[-1][0], 1104-shifts[-1][1]]
print(f"First position: x={first_position[0]}, y={first_position[1]}")
print(f"Last position: x={last_position[0]:.0f}, y={last_position[1]:.0f}")

First position: x=1150, y=1157
Last position: x=1264, y=1108


In [8]:
plot = False #@param {type:"boolean"}
print_results = True #@param {type:"boolean"}

object_position = np.empty((nbfiles, 2))

object_position[:,0] = np.linspace(first_position[0], last_position[0], nbfiles, endpoint=True)
object_position[:,1] = np.linspace(first_position[1], last_position[1], nbfiles, endpoint=True)

object_position[:,0] += shifts[:,0]
object_position[:,1] += shifts[:,1]

i = 0
corrected_object_position = []

if print_results: print("Asteroïd location:")

for file in os.listdir(PARENT_PATH):
    path = os.path.join(PARENT_PATH, file)
    if file.startswith("td10r"):
        corrected_object_position.append(
            image.find_object(
                path,
                Xcent=int(object_position[i,0]),
                Ycent=int(object_position[i,1]),
                box_size=20,
                plot=plot
            )
        )
        if print_results:
            print(f"    {file}: x={corrected_object_position[-1][0]:.3f}, y={corrected_object_position[-1][1]:.3f}")
        i += 1

Asteroïd location:
    td10r_n2_0096.fits: x=1149.476, y=1155.764
    td10r_n2_0098.fits: x=1153.065, y=1154.803
    td10r_n2_0100.fits: x=1156.795, y=1153.275
    td10r_n2_0102.fits: x=1160.082, y=1152.088
    td10r_n2_0104.fits: x=1163.119, y=1150.910
    td10r_n2_0106.fits: x=1166.214, y=1149.680
    td10r_n2_0108.fits: x=1098.138, y=1142.832
    td10r_n2_0110.fits: x=1101.293, y=1141.710
    td10r_n2_0112.fits: x=1106.845, y=1139.266
    td10r_n2_0114.fits: x=1109.833, y=1138.019
    td10r_n2_0116.fits: x=1108.365, y=1177.248
    td10r_n2_0118.fits: x=1111.691, y=1176.114
    td10r_n2_0123.fits: x=1162.941, y=1130.871
    td10r_n2_0125.fits: x=1166.305, y=1129.787
    td10r_n2_0127.fits: x=1169.671, y=1128.348
    td10r_n2_0129.fits: x=1172.986, y=1127.401
    td10r_n2_0131.fits: x=1177.555, y=1084.333
    td10r_n2_0133.fits: x=1181.321, y=1082.654
    td10r_n2_0135.fits: x=1150.937, y=1082.377
    td10r_n2_0137.fits: x=1154.124, y=1081.262
    td10r_n2_0144.fits: x=1176.226, y=114